In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install -qqq openai
!pip install -Uqqq chromadb
!pip install -q tiktoken
!pip install -q langchain
!pip install -q pypdf
!pip install -q accelerate
!pip install -U langchain-openai
!pip install -q PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [ ]:

import transformers
from transformers import AutoTokenizer
from langchain import HuggingFacePipeline
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders.pdf import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
import langchain
import chromadb
import torch
import accelerate

In [ ]:
import requests

def download_file_from_google_drive(file_id, destination):

    url = "https://drive.google.com/uc?id=" + file_id
    response = requests.get(url, stream=True)

    if response.status_code == 200:

        with open(destination, 'wb') as f:
            for chunk in response.iter_content(1024 * 1024):
                f.write(chunk)

        print("File downloaded successfully")
    else:
        print("Failed to download file")

file_id_list = ['1VIKVkxBXKwpfAieBrZ_OSTMfzhBjVoCb', '18yLsAli-G8erNjutq5YnJRUQlbH3LoMv', '1HfPb48GercUt2sm9Jcug1RzX1HmKNa9m'] # all there file given in email
destination_list = ['file1.pdf', 'file2.pdf', 'file3.pdf']

for file_id, destination in zip(file_id_list, destination_list):
    download_file_from_google_drive(file_id, destination)


In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-"

In [ ]:

def load_chunk_persist_pdf() -> Chroma:
    pdf_folder_path = "/content"
    documents = []
    for file in os.listdir(pdf_folder_path):
        if file.endswith('.pdf'):
            pdf_path = os.path.join(pdf_folder_path, file)

            loader = PyPDFLoader(pdf_path)
            documents.extend(loader.load())

    text_splitter = CharacterTextSplitter(separator = "\n", chunk_size=1000, chunk_overlap=10)
    chunked_documents = text_splitter.split_documents(documents)
    '''
    client = chromadb.Client()
    if client.list_collections(): OpenAIEmbeddings.create_collection("consent_collection")
    else:
        print("Collection already exists")
    '''
    vectordb = Chroma.from_documents(chunked_documents, OpenAIEmbeddings())
    return vectordb

In [ ]:
db = load_chunk_persist_pdf()

In [ ]:
db.similarity_search('', k=1)

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:

model = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
      "text-generation", #task
      model=model,
      tokenizer=tokenizer,
      torch_dtype=torch.bfloat16,
      trust_remote_code=True,
      device_map="auto",
      max_length=1000,
      do_sample=True,
      top_k=10,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id
)

llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
template = """
You are a intelligent assiatance where you have to guide the user on the bases of the question that are been aksed, you do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'.


Question - {question}

This is the context on which the bellow question is based - {context}

Answer:
"""


In [ ]:

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
print(
    prompt.format(
        question = "this is my question?",
        context = 'context'
    )
)


You are a intelligent assiatance where you have to guide the user on the bases of the question that are been aksed, you do not respond as 'User' or pretend to be 'User'. You only respond once as 'Assistant'.

Question - this is my question?

This is the context on which the bellow question is based - context

Answer:



In [ ]:

chain_type_kwargs = {"prompt": prompt}

In [ ]:
memory = ConversationBufferMemory(memory_key="history")

In [ ]:

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_type="similarity", search_kwargs={"k": 1}),
    chain_type_kwargs=chain_type_kwargs,
    memory = memory
)

In [ ]:
def chain_response(question):
  resp =  chain.invoke(question)
  return resp

In [ ]:
questions = [
    "What are the advantages and disadvantages of Limited Liability Corporation?",
    "What are the demand and supply schedules for sweatshirts?",
    "Differentiate between Chirs's LandScaping and Clear Lake Sporting Goods and tell which is better?",
    "Tell me about Reliance Industries Limited and what are the equity shares of Reliance Industries Limited?",
    "Define the profit and loss for the year ended 31st March,2013 of Mr. Sexena trading book?",
    "Difference between Branch Accounts and Departmental Accounts ?"
]


In [ ]:
def show_response(output):
    return output['result'].split('Answer:')[1].replace('\n', ' ')

In [ ]:
for q in questions:
    respn = show_response(chain_response(q))
    print(respn)

In [ ]:
while True:
  quest = str(input("User: "))
  print('Answer: ', show_response(chain_response(quest)))